In [7]:
import requests

def get_anime_episodes(anime_id):
    # Jikan API base URL for episodes
    url = f"https://api.jikan.moe/v4/anime/{anime_id}/episodes"
    episodes = []
    
    page = 1
    while True:
        response = requests.get(f"{url}?page={page}")
        if response.status_code != 200:
            print(f"Error: {response.status_code}")
            break
        
        data = response.json()
        episodes.extend(data.get("data", []))
        
        # Check if more pages exist
        if not data.get("pagination", {}).get("has_next_page"):
            break
        
        page += 1

    return episodes

# JJK anime ID on MAL is 40748
anime_id = 40748
episodes = get_anime_episodes(anime_id)

# Print episode titles
for episode in episodes:
    print(f"Episode {episode['mal_id']}: {episode['title']}")

Episode 1: Ryomen Sukuna
Episode 2: For Myself
Episode 3: Girl of Steel
Episode 4: Curse Womb Must Die
Episode 5: Curse Womb Must Die II
Episode 6: After Rain
Episode 7: Assault
Episode 8: Boredom
Episode 9: Small Fry and Reverse Retribution
Episode 10: Idle Transfiguration
Episode 11: Narrow-minded
Episode 12: To You, Someday
Episode 13: Tomorrow
Episode 14: Kyoto Sister School Exchange Event - Group Battle 0 -
Episode 15: Kyoto Sister School Exchange Event - Group Battle 1 -
Episode 16: Kyoto Sister School Exchange Event - Group Battle 2 -
Episode 17: Kyoto Sister School Exchange Event - Group Battle 3 -
Episode 18: Sage
Episode 19: Black Flash
Episode 20: Nonstandard
Episode 21: Jujutsu Koshien
Episode 22: The Origin of Blind Obedience
Episode 23: The Origin of Blind Obedience: 2
Episode 24: Accomplices


In [9]:
from bs4 import BeautifulSoup
import requests

def get_episode_characters(episode_url):
    # Fetch the episode page
    response = requests.get(episode_url)
    if response.status_code != 200:
        print(f"Failed to fetch the episode page: {response.status_code}")
        return None
    
    soup = BeautifulSoup(response.text, "html.parser")
    characters = []
    
    # Replace the selector below with the correct one from the webpage
    character_section = soup.find("div", {"class": "mw-parser-output"})
    if character_section:
        for item in character_section.find_all("a"):
            characters.append(item.text.strip())

    return characters

# Example: Episode 1 URL on Fandom Wiki
episode_url = "https://jujutsu-kaisen.fandom.com/wiki/Episode_1"
characters = get_episode_characters(episode_url)

if characters:
    print("Characters appearing in Episode 1:")
    for char in characters:
        print(char)
else:
    print("No character data found.")

Characters in Episode 1:
No characters found.
